In [4]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

import lightgbm as lgb

from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [5]:
df = pd.read_csv("../data/훈련데이터셋/3_훈련데이터셋_LAG_휴일포함.csv")
#df.info()

In [6]:
df.tail(5)

,날짜,전체코드,최저기온(°C),0.5m 지중온도(°C),평균 증기압(hPa),가조시간(hr),평균 상대습도(%),파워,lag_1d,lag_7d,휴일여부
233203,2024-01-24,1174011000,-11.2,2.3,1.7,10.1,52.0,115124.547,114960.859,108004.536,False
233204,2024-01-25,1174011000,-9.7,2.2,2.0,10.1,47.8,99907.500,115124.547,98758.864,False
233205,2024-01-26,1174011000,-6.9,2.1,3.4,10.1,62.5,94818.512,99907.500,NaN,False
233206,2024-01-27,1174011000,-4.6,2.0,4.0,10.2,70.0,86792.828,94818.512,76691.460,True
233207,2024-01-28,1174011000,-5.0,1.9,3.4,10.2,62.3,86389.477,86792.828,89640.934,True


In [7]:
feature_cols = [
    # 지역
    "전체코드",

    # 날씨
    
    "최저기온(°C)", "0.5m 지중온도(°C)", "평균 증기압(hPa)","가조시간(hr)","평균 상대습도(%)",

    # 기간

    "lag_1d", "lag_7d"

]
target_col = "파워"

In [8]:
new_df = df[["날짜"] + feature_cols + [target_col]].dropna().reset_index(drop=True)

In [9]:
# =========================
# 1) 시간순 Train/Valid/Test 분할
# =========================
new_df = new_df.sort_values("날짜").reset_index(drop=True)

n = len(new_df)
test_size = int(n * 0.20)
valid_size = int(n * 0.10)

train_end = n - (valid_size + test_size)
valid_end = n - test_size

train_df = new_df.iloc[:train_end]
valid_df = new_df.iloc[train_end:valid_end]
test_df  = new_df.iloc[valid_end:]

In [10]:
# =========================
# 2) X / y 분리
# =========================
drop_cols = [target_col, "날짜"]  # 날짜는 직접 입력 피처로 쓰지 않음(우린 파생변수로 이미 반영)
X_train = train_df.drop(columns=drop_cols)
y_train = train_df[target_col]

X_valid = valid_df.drop(columns=drop_cols)
y_valid = valid_df[target_col]

X_test = test_df.drop(columns=drop_cols)
y_test = test_df[target_col]

In [15]:

# =========================
# 3) 범주형 처리 (코드형 컬럼은 category로)
# =========================
cat_cols = []
for c in ["전체코드", "시군구코드", "법정동코드"]:
    if c in X_train.columns:
        cat_cols.append(c)

for c in cat_cols:
    X_train[c] = X_train[c].astype("category")
    X_valid[c] = X_valid[c].astype("category")
    X_test[c]  = X_test[c].astype("category")

In [16]:
# =========================
# 4) Xgboost 모델
# =========================
from xgboost import XGBRegressor
from sklearn.metrics import mean_squared_error, r2_score

xgb_model = XGBRegressor(
    n_estimators=1500,
    learning_rate=0.03,

    max_depth=5,
    min_child_weight=5,

    subsample=0.8,
    colsample_bytree=0.8,

    gamma=0.1,              # 불필요한 분기 억제
    reg_alpha=0.0,
    reg_lambda=5.0,         # L2 강화 (중요)

    objective="reg:squarederror",
    random_state=42,
    n_jobs=-1,
    tree_method="hist"
)

xgb_model.fit(X_train, y_train)

y_pred = xgb_model.predict(X_test)

rmse = mean_squared_error(y_test, y_pred, squared=False)
r2 = r2_score(y_test, y_pred)

print(f"[XGB] RMSE: {rmse:,.2f}")
print(f"[XGB] R2  : {r2:.4f}")

ValueError: DataFrame.dtypes for data must be int, float, bool or category. When categorical type is supplied, The experimental DMatrix parameter`enable_categorical` must be set to `True`.  Invalid columns:전체코드: category

In [ ]:
import pandas as pd

imp = pd.DataFrame({
    "feature": X_train.columns,
    "importance": xgb_model.feature_importances_
}).sort_values("importance", ascending=False)

imp.head(20)